# Pixel Intensity Analysis

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(evaluation_dir, 'pixel_intensity.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])
predictions.head()

## Compare the predictions to the ground truth

In [ ]:
lowpass = 0.08
highpass = 0.5

In [ ]:
import tqdm.auto as tqdm
import respiration.analysis as analysis

from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

scenario_results = []

for index, row in tqdm.tqdm(predictions.iterrows(), total=len(predictions)):
    subject, setting = row['subject'], row['setting']

    # Get the ground truth signal and sampling rate
    gt_signal = dataset.get_breathing_signal(subject, setting)

    prediction, sampling_rate = row['signal'], row['sampling_rate']

    pred_frequencies = analysis.sliding_window_analysis(prediction, sampling_rate, lowpass, highpass)

    min_length = min(len(prediction), len(gt_signal))

    gt_signal_cut = gt_signal[:min_length]
    gt_frequencies = analysis.sliding_window_analysis(gt_signal_cut, sampling_rate, lowpass, highpass)

    for metric in gt_frequencies:
        scenario_results.append({
            'subject': subject,
            'setting': setting,
            'model': row['method'],
            'metric': metric,
            'gt_frequencies': gt_frequencies[metric],
            'pred_frequencies': pred_frequencies[metric]
        })

scenario_results = pd.DataFrame(scenario_results)

In [ ]:
import itertools
import scipy.stats as stats

models = scenario_results['model'].unique()
metrics = scenario_results['metric'].unique()

evaluation_frequency = []

for (model, metric) in itertools.product(models, metrics):
    model_metric_df = scenario_results[
        (scenario_results['model'] == model) &
        (scenario_results['metric'] == metric)]

    # Concatenate the predicted and ground truth frequencies
    gt_frequencies = np.concatenate(model_metric_df['gt_frequencies'].values)
    pred_frequencies = np.concatenate(model_metric_df['pred_frequencies'].values)

    gt_frequencies = gt_frequencies[:len(pred_frequencies)]

    mae = np.mean(np.abs(gt_frequencies - pred_frequencies))
    rmse = np.sqrt(np.mean((gt_frequencies - pred_frequencies) ** 2))

    # Multiply the frequencies by 60 to get the bpm
    mae *= 60
    rmse *= 60

    corr, p = stats.pearsonr(gt_frequencies.flatten(), pred_frequencies.flatten())

    evaluation_frequency.append({
        'model': model,
        'metric': metric,
        'mae': mae,
        'rmse': rmse,
        'corr': corr,
        'p': p
    })

evaluation_frequency = pd.DataFrame(evaluation_frequency)

In [ ]:
evaluation_frequency